| [02_advanced/06_装饰器.ipynb](https://github.com/shibing624/python-tutorial/blob/master/02_advanced/06_装饰器.ipynb)  | Decorator装饰器  |[Open In Colab](https://colab.research.google.com/github/shibing624/python-tutorial/blob/master/02_advanced/06_装饰器.ipynb) |

# 装饰器：Decorator
如果你有一批变量想统一按一个规则处理，并且需要缩减代码，你需要函数。

如果你有一批函数想统一按一个规则处理，并且需要缩减代码，你需要装饰器（Decorator）

理清下面2点：

函数
- 接受参数
- 做点事情
- 返回结果


装饰器
- 接受函数作为参数
- 做点事情
- 返回一个函数


用 @ 来使用装饰器

使用 @ 符号来将某个函数替换为装饰符之后的函数：

例如这个函数：

In [25]:
def dec(f):
    print('I am decorating function', id(f))
    return f

In [26]:
def foo(x):
    print(x)  # I am decorating function 45206384

In [27]:
foo = dec(foo)

I am decorating function 140410365416640


可以替换为：

In [28]:
@dec
def foo(x):
    print(x)

I am decorating function 140410366082496


### 例子
定义两个装饰器函数，一个将原来的函数值加一，另一个乘二：

In [29]:
def plus_one(f):
    def new_func(x):
        return f(x) + 1

    return new_func

In [30]:
def times_two(f):
    def new_func(x):
        return f(x) * 2

    return new_func

定义函数，先乘二再加一：

In [31]:
@plus_one
@times_two
def foo(x):
    return int(x)

In [32]:
b = foo(2)
b  # 5

5

## 修饰器工厂
decorators factories 是返回修饰器的函数

它的作用在于产生一个可以接受参数的修饰器，

例如我们想将 函数 输出的内容写入一个文件去，可以这样做：

In [33]:
def super_loud(filename):
    fp = open(filename, 'w')

    def loud(f):
        def new_func(*args, **kw):
            fp.write(str(args))
            fp.writelines('\n')
            fp.write('calling with' + str(args) + str(kw))
            # 确保内容被写入
            fp.flush()
            fp.close()
            rtn = f(*args, **kw)
            return rtn

        return new_func

    return loud

In [44]:
@super_loud('test.txt')
def foo(x):
    print(x)


# 调用 foo 就会在文件中写入内容：
foo(100)

100


In [45]:
import os
os.remove('test.txt')

## @classmethod 装饰器
在 Python 标准库中，有很多自带的装饰器，

例如 classmethod 将一个对象方法转换了类方法：

In [35]:
class Foo(object):
    @classmethod
    def bar(cls, x):
        print('the input is', x)

    def __init__(self):
        pass

类方法可以通过 类名.方法 来调用：

In [36]:
Foo.bar(10)

the input is 10


## @property 装饰器
有时候，我们希望像 Java 一样支持 getters 和 setters 的方法，

这时候就可以使用 property 装饰器：

In [37]:
class Foo(object):
    def __init__(self, data):
        self.data = data

    @property
    def x(self):
        return self.data


此时可以使用 .x 这个属性查看数据（不需要加上括号）：

In [38]:
foo = Foo(22)
print(foo.x)

22


这样做的好处在于，这个属性是只读的：

foo.x = 1 会报错


如果想让它变成可读写，可以加上一个装饰符 @x.setter：

In [39]:
class Foo(object):
    def __init__(self, data):
        self.data = data

    @property
    def x(self):
        return self.data

    @x.setter
    def x(self, value):
        self.data = value

In [40]:
foo = Foo(1000)
foo.x

1000

In [42]:
foo.x = 2222
foo.x

2222

## 应用：定时器

要求：写一个定时器功能，要求监控一个执行程序，超时则报警。

如何完成？

In [55]:

import signal
import time


def set_timeout(num, callback):
    def wrap(func):
        def handle(signum, frame):  # 收到信号 SIGALRM 后的回调函数，参数1是信号的数字，参数2是the interrupted stack frame.
            raise RuntimeError

        def to_do(*args, **kwargs):
            try:
                signal.signal(signal.SIGALRM, handle)  # 设置信号和回调函数
                signal.alarm(num)  # 设置 num 秒的闹钟
                print('start alarm signal.')
                r = func(*args, **kwargs)
                print('close alarm signal.')
                signal.alarm(0)  # 关闭闹钟
                return r
            except RuntimeError as e:
                callback()

        return to_do

    return wrap


def after_timeout():  # 超时后的处理函数
    print("do something after timeout.")
    raise RuntimeError


@set_timeout(2, after_timeout)  # 限时 2 秒超时
def connect():  # 要执行的函数
    time.sleep(2.4)  # 函数执行时间，写大于2的值，可测试超时
    return "完成"

class Demo:
    @set_timeout(2, after_timeout)
    def conn(self):
        time.sleep(3)
        return "ok"

试一下：

In [56]:
try:
    a = connect()
    print(a)
except Exception as e:
    a = 'err'
    print(a)


start alarm signal.
do something after timeout.
err


In [58]:
b = Demo()
try:
    c = b.conn()
    print(c)
except RuntimeError as e:
    print('run time err.')

start alarm signal.
close alarm signal.
ok


如果不超时：

In [59]:
class Demo:
    @set_timeout(2, after_timeout)
    def conn(self):
        time.sleep(1)
        return "ok"
    
b = Demo()
try:
    c = b.conn()
    print(c)
except RuntimeError as e:
    print('run time err.')

start alarm signal.
close alarm signal.
ok


本节完。